In [20]:
import snscrape.modules.twitter as sntwitter

In [22]:
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction import _stop_words
from sklearn.metrics.pairwise import cosine_similarity as cosine
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import LatentDirichletAllocation
from nltk.tokenize.casual import casual_tokenize
import pandas as pd
from nlpia.data.loaders import get_data
from sklearn.decomposition import TruncatedSVD
import numpy as np

# Scraping data

## Scrape keywords "iphone user experience" & "ios user experience" 

In [49]:
twitter_dataset = pd.DataFrame(columns = ["text","label"])

In [50]:
#How many to grab
maxTweets = 1000

#Keyword search
keyword = 'iphone user experience' and 'ios user experience'
ios_data = sntwitter.TwitterSearchScraper(keyword + '-filter:links -filter:replies').get_items()

#Set the label for this group
label = "ios ux"
ios_tweets = []
#Collect in array and pair each tweet with a class label
for i,tweet in enumerate(ios_data):
    if i >= maxTweets :
        break    
    ios_tweets = ios_tweets + [[tweet.content, label]]

/var/folders/z5/t7l_xpd11rv10jj2p0x7qzl80000gn/T/ipykernel_4796/579595195.py:15: FutureWarning: content is deprecated, use rawContent instead
  ios_tweets = ios_tweets + [[tweet.content, label]]


In [53]:
ios_ux_tweets = twitter_dataset.append(pd.DataFrame(ios_tweets, columns = ["text", "label"]))

/var/folders/z5/t7l_xpd11rv10jj2p0x7qzl80000gn/T/ipykernel_4796/1423256215.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ios_ux_tweets = twitter_dataset.append(pd.DataFrame(ios_tweets, columns = ["text", "label"]))


In [54]:
ios_ux_tweets.head(10)

,text,label
0,It has been committed to providing a great use...,ios ux
1,AEDDON has been committed to providing a great...,ios ux
2,"Whenever a new mobile feature is launched, iOS...",ios ux
3,The renewed Xircus Web3 Protocol token ecosyst...,ios ux
4,"I have a working theory, I think lots of Andro...",ios ux
5,👩‍👩‍👧‍👦 Get everyone involved in the project t...,ios ux
6,Aeddon is a new Metaverse platform that aims t...,ios ux
7,"Dear @Solcasinoio,\n\n I am writing to expre...",ios ux
8,It has been committed to providing a great use...,ios ux
9,"Attention iOS Users, want to win a 100MH GPU?\...",ios ux


In [61]:
ios_ux_tweets.shape

(1000, 2)

In [55]:
file_name = "ios_ux_tweets"
ios_raw_tweets.to_csv("data/" + file_name + ".csv", sep = "\t", columns = ["text"], index = False)

## Scrape keywords "iphone complain"

In [56]:
#How many to grab
maxTweets = 1000

#Keyword search
keyword = 'iphone complain' 
ios_complain_data = sntwitter.TwitterSearchScraper(keyword + '-filter:links -filter:replies').get_items()

#Set the label for this group
label = "ios painpoint"
ios_complain_tweets = []
#Collect in array and pair each tweet with a class label
for i,tweet in enumerate(ios_complain_data):
    if i >= maxTweets :
        break    
    ios_complain_tweets = ios_complain_tweets + [[tweet.content, label]]

/var/folders/z5/t7l_xpd11rv10jj2p0x7qzl80000gn/T/ipykernel_4796/4087804531.py:15: FutureWarning: content is deprecated, use rawContent instead
  ios_complain_tweets = ios_complain_tweets + [[tweet.content, label]]


In [57]:
ios_complain_tweets = twitter_dataset.append(pd.DataFrame(ios_complain_tweets, columns = ["text", "label"]))

/var/folders/z5/t7l_xpd11rv10jj2p0x7qzl80000gn/T/ipykernel_4796/1185899651.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ios_complain_tweets = twitter_dataset.append(pd.DataFrame(ios_complain_tweets, columns = ["text", "label"]))


In [58]:
ios_complain_tweets.head(10)

,text,label
0,🧵\nYesterday VR YouTubers announced the upcomi...,ios painpoint
1,My teacher has still the @u2 songs of innocenc...,ios painpoint
2,Imagine if the requirements to complain about ...,ios painpoint
3,"okay but frfr, the only android defenders are ...",ios painpoint
4,Beware if you have Apple IPhone 14 .. Lock Scr...,ios painpoint
5,"""YoU'rE oN TwiTTeR eVeN tHoUgH ELoN mUsK oWnS ...",ios painpoint
6,I didn’t realize I hit a nerve on those that g...,ios painpoint
7,I've lost my phone (iphone 13) on 7th feb 2023...,ios painpoint
8,People complain about eggs being $7 but not a ...,ios painpoint
9,Karma banget dulu suka complain sama iPhone us...,ios painpoint


In [60]:
ios_complain_tweets.shape

(1000, 2)

In [59]:
file_name = "ios_complain_tweets"
ios_complain_tweets.to_csv("data/" + file_name + ".csv", sep = "\t", columns = ["text"], index = False)

# Clean tweets

## check duplicated tweets
some duplicated data should be moved but some may not.

In [63]:
ios_complain_dup = ios_complain_tweets[ios_complain_tweets.duplicated()==True]['text'].values
for i in ios_complain_dup:
    for j in ios_complain_dup:
        if i ==j:
            print(i,j)

Reddit users from all over the world complain about an unpleasant bug in iOS 16 firmware. When the automatic update function is enabled, the iPhone tries to install a new version of the software, but freezes in the process. In the morning, the owners discover that their Reddit users from all over the world complain about an unpleasant bug in iOS 16 firmware. When the automatic update function is enabled, the iPhone tries to install a new version of the software, but freezes in the process. In the morning, the owners discover that their
People who complain about their situation in life are the same ones that rush to buy the new iPhone as soon as it gets released… and they wonder why they’re stuck in the same position. People who complain about their situation in life are the same ones that rush to buy the new iPhone as soon as it gets released… and they wonder why they’re stuck in the same position.
People who complain about their situation in life are the same ones that rush to buy the

In [64]:
ios_ux_dup = ios_ux_tweets[ios_ux_tweets.duplicated()==True]['text'].values
for i in ios_ux_dup:
    for j in ios_ux_dup:
        if i ==j:
            print(i,j)

Frontend: HTML, CSS, JavaScript, React

Backend: Python, Node.Js, Ruby, Java, PHP

Database: Mysql, MongoDB, Oracle, PostgreSQL

Cloud: AWS, Azure, Google Cloud, Digital Ocean

UI/UX: Figma, Adobe XD, User Experience 

Which of these do you wish to learn? Follow us let’s help Frontend: HTML, CSS, JavaScript, React

Backend: Python, Node.Js, Ruby, Java, PHP

Database: Mysql, MongoDB, Oracle, PostgreSQL

Cloud: AWS, Azure, Google Cloud, Digital Ocean

UI/UX: Figma, Adobe XD, User Experience 

Which of these do you wish to learn? Follow us let’s help
Frontend: HTML, CSS, JavaScript, React

Backend: Python, Node.Js, Ruby, Java, PHP

Database: Mysql, MongoDB, Oracle, PostgreSQL

Cloud: AWS, Azure, Google Cloud, Digital Ocean

UI/UX: Figma, Adobe XD, User Experience 

Which of these do you wish to learn? Follow us let’s help Frontend: HTML, CSS, JavaScript, React

Backend: Python, Node.Js, Ruby, Java, PHP

Database: Mysql, MongoDB, Oracle, PostgreSQL

Cloud: AWS, Azure, Google Cloud, Digital

it seems the duplicated tweets of 'ios ux' and 'ios_complain' are not related to our research. So these have to be removed
#### They are removed not because they are duplicated, but because they are not related to the project!!

### Remove duplicated tweets

In [69]:
ios_ux_df = ios_ux_tweets.drop_duplicates(subset=['text'])
#reindex
ios_ux_df= ios_ux_df.reset_index(drop=True)
print(ios_ux_df)

                                                  text   label
0    It has been committed to providing a great use...  ios ux
1    AEDDON has been committed to providing a great...  ios ux
2    Whenever a new mobile feature is launched, iOS...  ios ux
3    The renewed Xircus Web3 Protocol token ecosyst...  ios ux
4    I have a working theory, I think lots of Andro...  ios ux
..                                                 ...     ...
979  Kevacoin mobile app iOS 6.7.5 and Android 8.2....  ios ux
980  Thanks for everyone who cheered me up lately, ...  ios ux
981  Due to the signup and KYC overload on our cust...  ios ux
982  #DeFi is complex, we get that. Smart contracts...  ios ux
983  So funny that many web developers believe that...  ios ux

[984 rows x 2 columns]


In [68]:
ios_complian_df = ios_complain_tweets.drop_duplicates(subset=['text'])
#reindex
ios_complian_df= ios_complian_df.reset_index(drop=True)
print(ios_complian_df)

                                                  text          label
0    🧵\nYesterday VR YouTubers announced the upcomi...  ios painpoint
1    My teacher has still the @u2 songs of innocenc...  ios painpoint
2    Imagine if the requirements to complain about ...  ios painpoint
3    okay but frfr, the only android defenders are ...  ios painpoint
4    Beware if you have Apple IPhone 14 .. Lock Scr...  ios painpoint
..                                                 ...            ...
983  Smashed my phone screen last night. Haven't do...  ios painpoint
984  I hate iPhone user that’s complain about their...  ios painpoint
985  using iPhone? - don't complain about petrol pr...  ios painpoint
986  Im tired of this I’m so oppressed mind state e...  ios painpoint
987  People complained that apple didn’t include 6E...  ios painpoint

[988 rows x 2 columns]
